##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys
def find_repo_root():
    try:
        script_path = os.path.abspath('.')
    except NameError:
        script_path = os.path.abspath(__file__)
    result = None
    while True:
        script_path = os.path.dirname(script_path)
        if os.path.isdir(os.path.join(script_path, '.git')):
            result = script_path
        if script_path == '/':
            return result
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)

## <i class="fa fa-wrench"></i> 安装必要依赖和工具

In [ ]:
import subprocess

libraries_to_install = [

]

tools_to_install = [
    'wget',
    'iputils-ping'
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")

# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-folder-plus"></i> 创建Content文件夹
创建Content文件夹来还原Colab的运行环境，以减少代码总体修改量

In [ ]:
import os

def check_environment(output):
    environments = {
        "AutoDL": {
            "content_path": "/root/autodl-tmp/content",
            "env_name": "AutoDL"
        },
        "OpenBayes": {
            "content_path": "/openbayes/home/content",
            "env_name": "OpenBayes"
        }
    }
    # 检查输出是否包含字符串"autodl"或"openbayes"
    # for i in output:
    if "AutoDL" in output:
        return environments["AutoDL"]
    elif "OpenBayes" in output:
        return environments["OpenBayes"]
    # 如果输出中没有包含上述字符串，则返回空字典
    return {}


def detect_environment():
    # 初始化 content_path 和 env_name 变量
    content_path = None
    env_name = None

    # 将命令行存储在列表中
    commands = [
        "cd /openbayes/home && chmod +x /etc/welcome && /etc/welcome",
        "chmod +x /etc/autodl-motd && /etc/autodl-motd"
    ]

    # 遍历命令行列表，执行命令并存储输出
    for command in commands:
        try:
            import subprocess
            # 使用 subprocess 模块执行命令
            r = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
            output = r.stdout
            # 将输出转换为字符串
            output = output.decode()
            # print(output)
            result = check_environment(output)
            content_path = result["content_path"]
            env_name = result["env_name"]
            # 如果检测到环境，则退出循环
            if content_path and env_name:
                break
        except Exception as e:
            # print("无法执行命令：", e)
            continue

    # 打印结果
    if content_path and env_name:
        print("当前运行环境：", env_name)
        print("内容路径：", content_path)
    else:
        print("未检测到当前运行环境")
        
    return {
        "content_path":content_path,
        "env_name":env_name
    }


content_path=detect_environment()["content_path"]
# 在根目录下创建Content文件夹
content_folder = content_path
if not os.path.exists(content_folder):
    os.makedirs(content_folder)

## <i class="fa fa-star"></i> Notebook环境切换
在JupyterLab的Notebook中使用新的Conda环境

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import ipywidgets as widgets

python_version = '3.8.11'

custom_env_name = 'DFRF' + python_version

# custom_env_name = 'base'

s = getoutput('conda info --env')
print(s)
if custom_env_name not in s:
    print(f'当前系统中没有{custom_env_name}环境，正在安装')
    # 创建Conda新的虚拟环境
    # 构建一个虚拟环境
    # 更新bashrc中的环境变量
    # 在 /openbayes/home 下创建一个新的环境：https://openbayes.com/docs/gear/conda/#%E4%BD%BF%E7%94%A8-conda-%E5%88%9B%E5%BB%BA%E6%96%B0%E7%9A%84%E7%8E%AF%E5%A2%83
    python_install_args = [
        'source /usr/local/etc/profile.d/conda.sh','&&',
        f'cd {root_dir}','&&',
        f'conda create -p {custom_env_name} python={python_version} -y','&&',
        'conda init bash','&&',
        'source /root/.bashrc -y'
    ]
    
    # !source /usr/local/etc/profile.d/conda.sh &&\
    # conda create -n $custom_env_name python=$python_version -y &&\
    # conda init bash && source /root/.bashrc -y
    
    python_install_cmd = (" ").join(python_install_args)
    !$python_install_cmd
    
else:
    print(f'已经安装了{custom_env_name}环境')

# clear_output(wait=True)
# print("Done!")

widgets.Button(
    value=False,
    description='环境部署成功!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

## 交代Base_args

In [ ]:
from func.env import setProxy,getWebUIDir
import subprocess

def find_conda_env(name):
    # Get a list of all conda environments on the system
    output = subprocess.run(["conda", "env", "list"], capture_output=True)
    envs = output.stdout.decode().split("\n")

    # Find the path of the environment
    for env in envs:
        if name in env:
            path = env.split()[-1]
            print(f"The path of the environment '{name}' is {path}")
            return path
    else:
        print(f"Environment '{name}' not found.")
        return None



try:
    proxy,region
except NameError:
    cb=setProxy()
    proxy=cb['proxy']
    region=cb['region']
    clear_output(wait=True)

webUIDir=getWebUIDir()
print(webUIDir)

custom_env_dir = find_conda_env(custom_env_name)

base_args = [
    'source activate','&&',
    f'conda activate {custom_env_dir}','&&',
    # proxy,'&&'
       ]

In [ ]:
pip_install_args = base_args[:]
pip_install_args += [
    'pip install torch==1.9.0 torchvision==0.10.0 pytorch3d','&&',
    f'cd {content_folder}','&&',
    # 'git clone https://github.com/facebookresearch/pytorch3d.git && cd pytorch3d && pip install -e .','&&',
    # 'git checkout d8f8bcb','&&'
    # 'git clone https://github.com/sstzal/DFRF','&&',
    'cd DFRF', '&&',
    'pip install -r requirements.txt'
    # '&&',
    # 'conda config --append channels conda-forge','&&',
    # 'conda install --yes --file requirements.txt','&&',
    # 'python -m pip install -r requirements_versions.txt'
]
pip_install_cmd = (" ").join(pip_install_args)
print(pip_install_cmd)
!$pip_install_cmd